Import Packages

In [421]:
import pandas as pd
import numpy as np
import warnings

from datetime import datetime, timedelta
from IPython.display import display
# !pip install -U pandasql
from pandasql import sqldf
mysql = lambda q: sqldf(q, globals())

warnings.filterwarnings("ignore")

Read Country Data

In [422]:
country_data_processed = pd.read_csv("Country_Processed_Table.csv")#read input data
country_data_processed.head()

,Country,Year,Country_Key,Continent,Region,Capital,Currency,Birth_Rate_Crude_Per_1000,Population_Total_Count,Death_Rate_Crude_Per_1000,GNI_Per_Capita_USD,Labour_Force_Female_Percent,Labour_Force_Total_Count,Age_Dependency_Ratio_Percent_Working_Age,Age_First_Marriage_Male,Age_First_Marriage_Female,Age_Dependency_Ratio_Old,Age_Dependency_Ratio_Young,Completeness_Birth_Registration_Percent
0,Canada,2005,0,North America,North America,Ottawa,CAD,10.6,32243753,7.1,34810,46.637376,17821886,44.495093,28.7,26.98,18.954219,25.540874,100.0
1,Canada,2006,1,North America,North America,Ottawa,CAD,10.9,32571174,7.0,38510,46.875172,18030705,44.236994,28.6,26.60,19.138346,25.098648,100.0
2,Canada,2007,2,North America,North America,Ottawa,CAD,11.2,32889025,7.1,43090,47.036713,18379605,44.034776,28.7,26.98,19.353443,24.681333,100.0
3,Canada,2008,3,North America,North America,Ottawa,CAD,11.3,33247118,7.2,45650,46.974754,18667574,43.933903,28.7,26.98,19.620978,24.312924,100.0
4,Canada,2009,4,North America,North America,Ottawa,CAD,11.3,33628895,7.1,43230,47.200377,18799155,43.976753,28.7,26.98,19.966597,24.010156,100.0


Create Fact Table and Add Month Data

In [423]:
fact_table = pd.DataFrame()

month_data_processed = pd.read_csv("Month_Processed_Table.csv")#read input data
fact_table = month_data_processed.merge(country_data_processed, on="Year")
fact_table = fact_table.sort_values('Country')

fact_table.head()


,Month_Key,Name,Month_Num,Quarter,Year,Decade,Country,Country_Key,Continent,Region,...,Death_Rate_Crude_Per_1000,GNI_Per_Capita_USD,Labour_Force_Female_Percent,Labour_Force_Total_Count,Age_Dependency_Ratio_Percent_Working_Age,Age_First_Marriage_Male,Age_First_Marriage_Female,Age_Dependency_Ratio_Old,Age_Dependency_Ratio_Young,Completeness_Birth_Registration_Percent
1491,165,October,10,4,2018,21,Bolivia,109,South America,South America,...,6.465,3840,39.209263,133000453,47.947297,26.675,22.44,8.665518,39.281779,71.9
1572,174,July,7,3,2019,21,Bolivia,110,South America,South America,...,6.510,4050,39.491377,136202238,47.635083,26.675,22.44,8.932369,38.702714,68.0
357,39,April,4,2,2008,20,Bolivia,99,South America,South America,...,6.849,1940,37.966295,110890081,51.508237,26.675,22.44,7.503345,44.004892,68.0
996,110,March,3,1,2014,21,Bolivia,105,South America,South America,...,6.437,3620,37.975372,122770335,49.357239,26.675,22.44,7.883945,41.473294,68.0
159,17,June,6,2,2006,20,Bolivia,97,South America,South America,...,7.087,1380,35.879524,103776784,52.414299,26.675,22.44,7.419868,44.994431,68.0


Connect Tables

In [424]:
population_data_processed = pd.read_csv("Pop_Processed_Table.csv")#read input data
education_processed = pd.read_csv("Education_Processed_Table.csv")#read input data
health_processed = pd.read_csv("Health_Dimension.csv")#read input data
quality_life_processed = pd.read_csv("QualityOfLife_Dimension.csv")

fact_table = fact_table.merge(education_processed, left_on=['Country','Year'], right_on=['Country_Ref','Year_Ref'], how='left').drop(columns=["Country_Ref", "Year_Ref"], axis=1)
fact_table = fact_table.merge(population_data_processed, left_on=['Country','Year'], right_on=['Country_Ref','Year_Ref'], how='left' ).drop(columns=["Country_Ref", "Year_Ref"], axis=1)
fact_table = fact_table.merge(health_processed, left_on=['Country','Year'], right_on=['Country_Ref','Date'], how='left').drop(columns=["Country_Ref", "Date"], axis=1)
fact_table = fact_table.merge(quality_life_processed, left_on=['Country','Year'], right_on=['Country_Ref','Date'], how='left').drop(columns=["Country_Ref", "Date"], axis=1)


events_processed = pd.read_csv("Event_Pro_Table.csv")#read input data

query = '''
Select * 
FROM fact_table 
left join events_processed

on
fact_table.Country = events_processed.Country_Name
and fact_table.Month_Num >= events_processed.Start_Month
and fact_table.Month_Num <= events_processed.End_Month
and fact_table.Year >= events_processed.Start_Year
and fact_table.Year <= events_processed.End_Year
'''

fact_table = mysql(query)

fact_table['Event_Key'] = fact_table['Event_Key'].fillna(-1)
fact_table['Event_Key'] = fact_table['Event_Key'].astype(int)

fact_table["Fact_Key"] = np.arange(len(fact_table))

fact_table = pd.DataFrame(fact_table, columns=["Fact_Key","Month_Key", "Country_Key", "Population_Key", "Education_Key", "Health_Key", "QOL_Key", "Event_Key", "Country", "Year", "Name", "Month_Num"])

fact_table.head(70)

,Fact_Key,Month_Key,Country_Key,Population_Key,Education_Key,Health_Key,QOL_Key,Event_Key,Country,Year,Name,Month_Num
0,0,165,109,109,13,13,13,-1,Bolivia,2018,October,10
1,1,174,110,110,14,14,14,-1,Bolivia,2019,July,7
2,2,39,99,99,3,3,3,-1,Bolivia,2008,April,4
3,3,110,105,105,9,9,9,-1,Bolivia,2014,March,3
4,4,17,97,97,1,1,1,-1,Bolivia,2006,June,6
...,...,...,...,...,...,...,...,...,...,...,...,...
65,65,159,109,109,13,13,13,-1,Bolivia,2018,April,4
66,66,122,106,106,10,10,10,-1,Bolivia,2015,March,3
67,67,27,98,98,2,2,2,63,Bolivia,2007,April,4
68,68,158,109,109,13,13,13,-1,Bolivia,2018,March,3


Add Measures

In [425]:
population_human_development_raw = pd.read_csv("human_development_index_raw.csv")#read input data
QOL = pd.read_csv("QOL Measures.csv")#read input data

countries = ["Canada", "United States", "Mexico", "Burundi", "Chad", "Niger", "Bolivia", "Philippines", "Indonesia"]

population_human_development_processed = population_human_development_raw[population_human_development_raw["Entity"].isin(countries)]
population_human_development_processed = population_human_development_processed[population_human_development_processed["Year"] >2004]
population_human_development_processed = population_human_development_processed.drop(["Code"], axis=1)

population_human_development_processed.columns =["Entity", "Year_Ref", "HDI"]

country = "Canada"
for country in countries:
    for i in range(3):
        country_data = {}
        country_data["Entity"] = country
        country_data["Year_Ref"] = 2018+i
        population_human_development_processed = population_human_development_processed.append(pd.DataFrame([country_data]), ignore_index=True)


population_human_development_processed['Entity'].replace({"United States": "USA"}, inplace = True)
population_human_development_processed = population_human_development_processed.sort_values(["Entity","Year_Ref"], ascending=[True,True])
population_human_development_processed.fillna(method="ffill", inplace=True)#perhaps change to extrapolation

fact_table = fact_table.merge(population_human_development_processed, left_on=['Country','Year' ], right_on=['Entity','Year_Ref'], how='left').drop(columns=["Entity", "Year_Ref"])
fact_table = fact_table.merge(QOL, left_on=['Country','Year' ], right_on=['Entity','Year_Ref'], how='left').drop(columns=["Entity", "Year_Ref"])

def label_race (row):
   if row["HDI"]==5 :
      return 5
   elif row['HDI'] >0.9 :
      return 1
   elif row['HDI'] > 0.8 :
      return 2
   elif row['HDI'] > 0.6 :
      return 3
   else:
      return 4

fact_table["HDI_Measure"] = fact_table.apply(lambda row: label_race(row), axis=1)

def label_race (row):
   if row["QOL"]>160 :
      return 1
   elif row['QOL'] >130 :
      return 2
   elif row['QOL'] > 100 :
      return 3
   elif row['QOL'] > 80 :
      return 4
   else:
      return 5

fact_table["QOL_Measure"] = fact_table.apply(lambda row: label_race(row), axis=1)


def label_race (row):
   if row["HDI"]>0.8 :
      return 1
   elif row['HDI'] >0.5 :
      return 2
   else:
      return 3

fact_table["Dev_Measure"] = fact_table.apply(lambda row: label_race(row), axis=1)

fact_table = fact_table.sort_values(["Country","Year"], ascending=[True,True])
fact_table = fact_table.drop(columns=["HDI", "QOL", "Country", "Year", "Name", "Month_Num"])
display(fact_table.head(50))
print(fact_table.shape)

,Fact_Key,Month_Key,Country_Key,Population_Key,Education_Key,Health_Key,QOL_Key,Event_Key,HDI_Measure,QOL_Measure,Dev_Measure
114,114,6,96,96,0,0,0,-1,3,4,2
117,117,2,96,96,0,0,0,-1,3,4,2
125,125,7,96,96,0,0,0,-1,3,4,2
143,143,4,96,96,0,0,0,-1,3,4,2
144,144,3,96,96,0,0,0,-1,3,4,2
152,152,5,96,96,0,0,0,-1,3,4,2
158,158,8,96,96,0,0,0,-1,3,4,2
166,166,10,96,96,0,0,0,-1,3,4,2
167,167,0,96,96,0,0,0,-1,3,4,2
176,176,11,96,96,0,0,0,-1,3,4,2


(1738, 11)


Output to csv for db loading

In [426]:
fact_table.to_csv('Fact_Table.csv', index = False, header=True)